# RAG 基础架构
> By SherryAGI | Retrieval-Augmented Generation 入门

## 什么是 RAG？

**Retrieval-Augmented Generation** = 检索增强生成

```
用户提问 → 检索相关文档 → 结合文档生成答案
```

### 为什么需要 RAG？

| 问题 | 纯 LLM | RAG |
|------|--------|-----|
| 知识过时 | ❌ 训练数据截止 | ✅ 实时检索最新文档 |
| 私有数据 | ❌ 无法访问 | ✅ 连接企业知识库 |
| 幻觉问题 | ❌ 可能编造 | ✅ 基于真实文档 |
| 来源追溯 | ❌ 无法验证 | ✅ 引用原文出处 |

In [ ]:
# 安装依赖
# pip install langchain langchain-openai chromadb

import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings

print('🚀 RAG 基础架构 by SherryAGI')

## RAG 核心流程

```
1. 文档加载 (Load)
   ↓
2. 文本分块 (Split)
   ↓
3. 向量化 (Embed)
   ↓
4. 存储 (Store)
   ↓
5. 检索 (Retrieve)
   ↓
6. 生成 (Generate)
```

## 1. 文档加载

In [ ]:
# 示例：企业知识文档
documents = [
    {
        "content": """公司报销政策：
        1. 差旅费：经济舱机票、三星级酒店
        2. 餐饮费：每日上限 100 元
        3. 交通费：公共交通或打车（需发票）
        4. 审批流程：500元以下直接报销，500元以上需主管审批""",
        "metadata": {"source": "HR政策", "category": "报销"}
    },
    {
        "content": """请假制度：
        1. 年假：入职满1年享有5天，每增加1年加1天，上限15天
        2. 病假：需提供医院证明，每年累计不超过30天
        3. 事假：需提前3天申请，每年累计不超过10天
        4. 审批：3天以内主管审批，3天以上需HR审批""",
        "metadata": {"source": "HR政策", "category": "请假"}
    },
    {
        "content": """IT 支持流程：
        1. 提交工单：通过内部系统提交
        2. 响应时间：普通问题24小时，紧急问题4小时
        3. 设备申请：需部门主管审批
        4. 密码重置：可自助或联系 IT 热线 8888""",
        "metadata": {"source": "IT手册", "category": "支持"}
    }
]

print(f'加载了 {len(documents)} 个文档')

## 2. 文本分块

In [ ]:
# 分块策略
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,      # 每块最大字符数
    chunk_overlap=50,    # 块之间重叠字符数
    separators=["\n\n", "\n", "。", "，", " "]
)

# 分块
chunks = []
for doc in documents:
    splits = text_splitter.split_text(doc["content"])
    for split in splits:
        chunks.append({
            "content": split,
            "metadata": doc["metadata"]
        })

print(f'分块后共 {len(chunks)} 个片段')
for i, chunk in enumerate(chunks[:3]):
    print(f'\n--- 片段 {i+1} ---')
    print(chunk["content"][:100] + '...')

## 3. 向量化 & 存储

In [ ]:
# 使用 OpenAI Embeddings（需要 API Key）
# os.environ["OPENAI_API_KEY"] = "your-api-key"

# 或使用本地模型（推荐企业使用）
# from langchain_community.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-zh-v1.5")

# 示例：模拟向量化
import numpy as np

def mock_embed(text):
    """模拟向量化（实际使用时替换为真实 Embedding 模型）"""
    np.random.seed(hash(text) % 2**32)
    return np.random.rand(384).tolist()

# 向量化所有片段
for chunk in chunks:
    chunk["embedding"] = mock_embed(chunk["content"])

print(f'向量维度: {len(chunks[0]["embedding"])}')

## 4. 检索

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def search(query, top_k=3):
    """语义搜索"""
    query_embedding = mock_embed(query)
    
    # 计算相似度
    similarities = []
    for chunk in chunks:
        sim = cosine_similarity([query_embedding], [chunk["embedding"]])[0][0]
        similarities.append((sim, chunk))
    
    # 排序返回 Top K
    similarities.sort(key=lambda x: -x[0])
    return similarities[:top_k]

# 测试检索
query = "出差报销标准是什么？"
results = search(query)

print(f'🔍 查询: {query}\n')
for i, (score, chunk) in enumerate(results):
    print(f'--- 结果 {i+1} (相似度: {score:.3f}) ---')
    print(f'来源: {chunk["metadata"]["source"]}')
    print(chunk["content"][:150])
    print()

## 5. 生成答案

In [ ]:
def generate_answer(query, context_chunks):
    """基于检索结果生成答案（示例）"""
    
    # 构建 Prompt
    context = "\n\n".join([c["content"] for _, c in context_chunks])
    
    prompt = f"""基于以下企业知识库内容回答问题。如果无法从内容中找到答案，请说明。

知识库内容：
{context}

问题：{query}

答案："""
    
    # 实际使用时调用 LLM
    # response = llm.invoke(prompt)
    
    print('📝 生成的 Prompt:')
    print(prompt)
    print('\n💡 提示：将此 Prompt 发送给 LLM 即可获得答案')

generate_answer(query, results)

## 6. 完整 RAG Pipeline

In [ ]:
class SimpleRAG:
    """简单 RAG 实现 by SherryAGI"""
    
    def __init__(self):
        self.chunks = []
    
    def add_document(self, content, metadata=None):
        """添加文档"""
        chunk = {
            "content": content,
            "metadata": metadata or {},
            "embedding": mock_embed(content)
        }
        self.chunks.append(chunk)
    
    def query(self, question, top_k=3):
        """查询"""
        results = search(question, top_k)
        return {
            "question": question,
            "sources": [c["metadata"].get("source", "未知") for _, c in results],
            "context": [c["content"] for _, c in results]
        }

# 使用示例
rag = SimpleRAG()
for doc in documents:
    rag.add_document(doc["content"], doc["metadata"])

result = rag.query("年假有多少天？")
print(f'问题: {result["question"]}')
print(f'来源: {result["sources"]}')

---
**By SherryAGI** | [DigitalTrans](https://github.com/AIB612/DigitalTrans)